In [1]:
%matplotlib inline

import sys

sys.path.append("..")

In [2]:
import os, pickle, sys, torch

from settings import ROOT_DIR

resume = '19-05-10T12-04'
start_epoch = 1

checkpoint_directory = os.path.join(ROOT_DIR, 'static', 'checkpoints', 'triplet_cnn', resume)

net = torch.load(os.path.join(checkpoint_directory, '_net_%s.pth' % start_epoch))
net

TripletNetwork(
  (embedding_network): ConvolutionalNetwork(
    (convolution_1): Conv2d(3, 6, kernel_size=(5, 5), stride=(1, 1))
    (convolution_2): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
    (convolution_3): Conv2d(16, 20, kernel_size=(4, 4), stride=(1, 1))
    (fully_connected_1): Linear(in_features=16820, out_features=15000, bias=True)
    (fully_connected_2): Linear(in_features=15000, out_features=1200, bias=True)
    (fully_connected_3): Linear(in_features=1200, out_features=125, bias=True)
  )
)

In [6]:
from torch.utils.data.dataloader import default_collate

from src.datasets import get_dataset

dataset_name = 'sketchy_test_photos_triplets'
workers = 4
collate = default_collate

dataset = get_dataset(dataset_name)

batch_size = 32

data_loader = torch.utils.data.DataLoader(
    dataset, batch_size=batch_size, shuffle=True,
    num_workers=workers, collate_fn=collate
)

print(sys.getsizeof(dataset))
print(sys.getsizeof(data_loader))

56
56


In [7]:
real_batch = next(iter(data_loader))

print(sys.getsizeof(real_batch))

112


In [8]:
anchors, positives, negatives = real_batch

anchor_embedding, positive_embedding, negative_embedding, distance_to_positive, distance_to_negative = net(anchors[0], positives[0], negatives[0])

In [9]:
print(distance_to_positive, distance_to_negative)

tensor([1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05], grad_fn=<NormBackward1>) tensor([1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05, 1.1180e-05,
        1.1180e-05, 1.1180e-05], grad_fn=<NormBackward1>)


In [11]:
from torch import nn

loss = nn.MarginRankingLoss(margin=margin, reduction='mean')

target = -torch.ones(anchors[1].size()[0])

triplet_loss = loss(distance_to_positive, distance_to_negative, target) # = d2positive - d2negative + margin

triplet_loss

NameError: name 'loss_fn' is not defined

In [ ]:
def KNN(mat, k):
    mat = mat.float()
    mat_square = torch.mm(mat, mat.t())
    diag = torch.diagonal(mat_square)
    diag = diag.expand_as(mat_square)
    dist_mat = (diag + diag.t() - 2 * mat_square)
    dist_col = dist_mat[-1, :-1]
    # k+1 because the nearest must be itself
    val, index = dist_col.topk(k, largest=False, sorted=True)
    return val, index


a = KNN(query_vectors, 20)